# 1. 엑셀데이터 불러오기

In [113]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import numpy as np
import re


df_input = pd.read_csv('phone_info.csv')
df_input.iloc[0]

pl_id                   1
pl_maker            apple
pl_group            apple
pl_model_code       A2221
pl_name             아이폰11
pl_model_name    iphone11
Name: 0, dtype: object

# 2. 리뷰크롤링

In [51]:
import time
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                      #스펙테이블 아이디
    "pl_model_code": "",              #모델코드
    "pl_name": "",                    #모델영문명
    
    "star": "",                       #별점
    "market": "",                     #구입처(ex 인터파크, 11번가)
    "write_id": "",                   #작성자(일부가림처리)
    "upload_day": "",                 #업로드날짜
    "title":"",                       #리뷰제목 
    "text": ""                        #리뷰내용 
}

crawl_data = pd.DataFrame(my_dict,index=[0])
crawl_data_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

def search_by_category(df_input,crawl_data,index,option):
    if option == "skt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000246&frm=NVSHMDL&iq='
    elif option == "kt":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000247&frm=NVSHMDL&iq='
    elif option == "lg":
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000248&frm=NVSHMDL&iq='
    else:
        url1 = 'https://search.shopping.naver.com/search/category?catId=50000204&frm=NVSHMDL&iq='
    url2 = '&origQuery&pagingIndex=1&pagingSize=40&productSet=model&query&sort=rel&timestamp=&viewType=list'
    driver.get(url1+df_input.iloc[index][4]+url2)
    driver.implicitly_wait(3)
    
    
    #페이지에 있는 링크 다 긁어오기, 일단 스크롤기능 추가안해서 5개만 가져옴 => 5개 넘어가면 검색정확도 떨어져서 고정
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')
    div_list = soup.select(".basicList_info_area__17Xyo")
    
    link_list = []
    for i in range(len(div_list)):
        if div_list[i].select("div.basicList_info_area__17Xyo > div.basicList_etc_box__1Jzg6 > a") == []:
            pass
        else:
            link_list.append(div_list[i].select(".basicList_link__1MaTN")[0])
    

    for link in link_list:
        url = 'window.open("' + link.get('href') +'");'
        driver.execute_script(url)
        driver.switch_to.window(driver.window_handles[-1])
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')
        review_list = soup.select(".reviewItems_list_review__1sgcJ")
        review_list = review_list[0].find_all("li")
        
        for review in review_list:
            area1 = review.select(".reviewItems_etc_area__2P8i3")
            area1 = area1[0].find_all("span")
            
            star = area1[0].text.replace("평점","") 
            market = area1[3].text 
            write_id = area1[4].text 
            upload_day = area1[5].text 
            
            area2 = review.select(".reviewItems_review_text__2Bwpa")
            title = area2[0].select(".reviewItems_title__39Z8H")[0].text
            text = area2[0].select(".reviewItems_text__XIsTc")[0].text
            
            my_dict = {
                "pl_id": df_input.iloc[index][0],              #기종아이디(자체부여한것)
                "pl_model_code": df_input.iloc[index][3],      #모델코드
                "pl_name": df_input.iloc[index][4],            #모델영문명

                "star": star,                       #별점
                "market": market,                   #구입처(ex 인터파크, 11번가)
                "write_id": write_id,               #작성자(일부가림처리)
                "upload_day": upload_day,           #업로드날짜
                "title":title,                      #리뷰제목 
                "text": text,                       #리뷰내용 
            }
            
            if my_dict is None or my_dict == None:
                pass
            else:
                a = pd.DataFrame(my_dict,index=[0])
                crawl_data = crawl_data.append(a)
        
        # 현재 탭 닫기
        driver.close()
        # 맨 처음 탭으로 변경(0번 탭)
        driver.switch_to.window(driver.window_handles[0])
    #display(crawl_data)
    return crawl_data

# 초기화 --------------------------------------------
chrome_options = Options()
chrome_options.add_argument("--headless")
driver  = webdriver.Chrome('../chromedriver')
driver.implicitly_wait(3)


for i in range(150,180):
#     print("=================",i,"skt","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"skt")) 
#     print("=================",i,"kt","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"kt"))
#     print("=================",i,"lg","=================")
#     crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"lg"))
    crawl_data = crawl_data.append(search_by_category(df_input,crawl_data_none,i,"all"))
    
crawl_data = crawl_data[crawl_data['pl_id'] != '']
crawl_data = crawl_data.reset_index(drop=True)

    
    
    

In [52]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 100)
display(crawl_data)

pandas version:  1.0.5


,pl_id,pl_model_code,pl_name,star,market,write_id,upload_day,title,text
1,152,LG-F540,볼트,4,,jsmj****,11.08.08.,물 흐리네,이거 싱글코어 입니다. 누가 듀얼이래요...그리고 이건 글로벌모델이라 스펙 상승 없...
2,152,LG-F540,볼트,3,,qwer****,11.04.12.,왜 스펙이 안써있냐,"CPU,RAM 뭔데.."
3,152,LG-F540,볼트,5,,sh38****,11.03.30.,ㄷ,그동안에 HTC 핸드폰을 보면 스팩이 어마어마 했었는데 이건 4G 라는 단어가 사람...
4,152,LG-F540,볼트,1,,ywol****,11.02.22.,이거 스펙이 모임??,"2.3진져 브래드탑재..1.2GHz 듀얼코어 탑재,전면카메라 500메가.이건데 당신..."
5,152,LG-F540,볼트,5,,you0****,11.02.22.,4G 폰이라는 것에 의미를 둡시다.,퀄컴듀얼코어1.2Ghz(MSM8655 1Ghz+MDM9600)전면130.500은루...
6,152,LG-F540,볼트,1,,zero****,11.02.22.,엑스큐브//// 모르면 말을 하지 마세요.,아직도 루머스펙을 믿고 있다니ㄷㄷ HTC홈페이지에 정식으로 공개된 스펙인데요??
7,152,LG-F540,볼트,5,,vksa****,11.03.01.,나오기는 커녕,아직 출시예정입니다두고두고 지켜봐야지
8,152,LG-F540,볼트,1,,sumo****,11.01.18.,핵심을 빼먹은 엉터리 평가로군요,세상에 무슨 이런 평이 다 있나요?하드웨어 스펙에서 가장 중요한 CPU와 디스플...
9,152,LG-F540,볼트,1,,hent****,11.01.19.,허접하고 정보가 엉망인 기사를 인용한게 웃기네..,"작년 HTC 티저에서 이미 전면캠 500만,배터리 1800,내장Mem 16G,CPU..."
10,152,LG-F540,볼트,5,,fast****,11.01.12.,내가 가지고 싶던 바로 그폰 /,"최고의 스펙을 자랑하고 있는, 썬더볼트~ 아직 출시전이지만 벌써부터 유명한거 아..."


In [53]:
crawl_data.to_csv('review_data1.csv.csv',encoding='utf-8-sig')

# 엑셀에서 pl_id 오름차순, text_len(추가후) 내림차순 정렬하고 난 후 => file.csv

In [108]:
#####################################################################################
######################### 크롤링 데이터 초기화 ######################################
#####################################################################################
my_dict = {
    "pl_id": "",                    #모델영문명
    "text1": "",
    "text2": "",
    "text3": ""
}

filtered_reivew = pd.DataFrame(my_dict,index=[0])
filtered_reivew_none = pd.DataFrame(my_dict,index=[0])
#######################################################################################
#######################################################################################
#######################################################################################

df_output = pd.read_csv('file.csv')
pl_ids = df_output['pl_id'].unique()
models = []
for pl_id in pl_ids:
    is_name = df_output['pl_id'] == pl_id
    model = df_output[is_name]
    my_dict = {
    "pl_id": pl_id,                 
    }
    
    print(len(model))
    if len(model) > 0:
        my_dict["text1"] = model.iloc[0]['text']
    if len(model) > 1:
        my_dict["text2"] = model.iloc[1]['text']
    if len(model) > 2:
        my_dict["text3"] = model.iloc[2]['text']
    
    filtered_reivew = filtered_reivew.append(pd.DataFrame(my_dict,index=[0]))

89
67
60
80
44
72
77
76
43
86
22
62
70
10
68
8
20
21
20
42
17
15
20
3
25
22
29
45
14
26
44
100
55
14
57
100
10
8
8
2
4
8
23
1
77
80
88
81
89
74
80
39
81
54
42
82
100
85
13
13
36
33
100
85
37
32
25
7
71
12
1
56
16
92
12
6
20
45
86
32
20
20
10
10
1
10
7
11
4
13
10
27
1
8
8
8
4
1
2
63
35
45
21
42
31
16
3
20
20
20
3
3
5
1
1
10
10
5
4
48
6
52
65
55
28
23
20
20
21
21
20
49
29
20
22
1
35
35
35
35
4
13
100
20
20
15
16
12
3
20


In [110]:
filtered_reivew.to_csv('filtered_reivew.csv',encoding='utf-8-sig')